# Tornado Property Damage Prediction

## Label: Total Property Damage
## Features: Tornado Scale, Length, Width, Duration [in minutes]

### Import libraries

In [1]:
# Load all the necessary libraries to ingest and transform the dataset
import pandas as pd
import numpy as np
import scipy as sp
import datetime
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from urllib.request import urlopen
from zipfile import ZipFile

### Scape website for target file list

In [2]:
# Store URL for the Storm Events page of the NOAA website 
urlLocation = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/"

In [3]:
urlLocation

'https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/'

In [4]:
# Read the webpage
r = urlopen(urlLocation)
# Identify the target string
goodlines = 'StormEvents_details-ftp'
# Initialize the file list variable
fileList = []
# Loop through each line of the webpage
for line in r:
    # Converts each line to a str datatype
    urlLine = str(line)
    # Determines if the target string appears in the current line
    if goodlines in urlLine:
        # Split line into three parts with target file at index 1
        urlLine = re.split('.csv.gz">|</a></td><td align="right">', urlLine)
        # Add the name of the file at index 1 to the file list variable
        fileList.append(urlLine[1])

# Show number of files in the list
print(len(fileList))


73


### Build the DataFrame by looping through each .gzip file and extracting the .csv

In [5]:
# Initialize file counter
fileCounter = 0
# Loop through file list, extract data and load to a dataframe
for file in fileList:
    targetFile = urlLocation + file
    if fileCounter == 0:
        dfWeather = pd.read_csv(targetFile, compression='gzip', low_memory=False)
        #dfWeather
    else:
        dfTemp = pd.read_csv(targetFile, compression='gzip', low_memory=False)
        dfWeather = pd.concat([dfWeather, dfTemp])
    fileCounter = fileCounter + 1

# Determine row count    
len(dfWeather)

1778844

### Transform the dataset

In [6]:
# Determine column number and datatype
dfWeather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1778844 entries, 0 to 53829
Data columns (total 51 columns):
 #   Column              Dtype  
---  ------              -----  
 0   BEGIN_YEARMONTH     int64  
 1   BEGIN_DAY           int64  
 2   BEGIN_TIME          int64  
 3   END_YEARMONTH       int64  
 4   END_DAY             int64  
 5   END_TIME            int64  
 6   EPISODE_ID          float64
 7   EVENT_ID            int64  
 8   STATE               object 
 9   STATE_FIPS          float64
 10  YEAR                int64  
 11  MONTH_NAME          object 
 12  EVENT_TYPE          object 
 13  CZ_TYPE             object 
 14  CZ_FIPS             int64  
 15  CZ_NAME             object 
 16  WFO                 object 
 17  BEGIN_DATE_TIME     object 
 18  CZ_TIMEZONE         object 
 19  END_DATE_TIME       object 
 20  INJURIES_DIRECT     int64  
 21  INJURIES_INDIRECT   int64  
 22  DEATHS_DIRECT       int64  
 23  DEATHS_INDIRECT     int64  
 24  DAMAGE_PROPERTY     object

In [7]:
# Check the first few rows
dfWeather.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,40.0,...,0.0,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,48.0,...,0.0,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,NaN,PUB
2,195007,5,1800,195007,5,1800,NaN,10104927,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,NaN,PUB
3,195007,5,1830,195007,5,1830,NaN,10104928,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,NaN,PUB
4,195007,24,1440,195007,24,1440,NaN,10104929,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,NaN,PUB


In [8]:
# Get some basic statistics on the numeric columns
dfWeather.describe()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE_FIPS,YEAR,...,CATEGORY,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_FIPS,BEGIN_RANGE,END_RANGE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
count,1.778844e+06,1.778844e+06,1.778844e+06,1.778844e+06,1.778844e+06,1.778844e+06,1.546598e+06,1.778844e+06,1.778843e+06,1.778844e+06,...,445.000000,269483.000000,269483.000000,2861.000000,982098.000000,981823.000000,1.098283e+06,1.098275e+06,920794.000000,920787.000000
mean,2.006136e+05,1.497588e+01,1.316380e+03,2.006136e+05,1.635981e+01,1.481522e+03,5.732159e+05,3.297802e+06,3.191975e+01,2.006076e+03,...,1.548315,0.920223,34.829871,103.432716,2.369925,2.316897,3.785435e+01,-9.079009e+01,37.905419,-90.556328
std,1.222359e+03,9.075370e+00,6.695126e+02,1.222358e+03,9.087049e+00,6.055888e+02,1.549765e+06,3.962400e+06,1.750862e+01,1.222388e+01,...,1.039979,8.044942,133.181185,84.046391,5.916679,4.630372,4.941089e+00,1.032798e+01,4.969348,10.580593
min,1.950010e+05,1.000000e+00,0.000000e+00,1.950010e+05,1.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,1.950000e+03,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-1.440000e+01,-1.714000e+02,-14.456000,-171.400000
25%,2.000060e+05,7.000000e+00,8.460000e+02,2.000060e+05,9.000000e+00,1.150000e+03,6.793225e+04,4.700018e+05,1.900000e+01,2.000000e+03,...,1.000000,0.000000,0.000000,47.000000,0.000000,0.000000,3.443570e+01,-9.748995e+01,34.590000,-97.370000
50%,2.008060e+05,1.500000e+01,1.510000e+03,2.008060e+05,1.600000e+01,1.612000e+03,1.358000e+05,9.389755e+05,3.100000e+01,2.008000e+03,...,1.000000,0.000000,0.000000,91.000000,1.000000,1.000000,3.807000e+01,-9.105000e+01,38.140000,-90.500000
75%,2.015060e+05,2.300000e+01,1.817000e+03,2.015060e+05,2.400000e+01,1.908000e+03,1.144955e+06,5.510718e+06,4.600000e+01,2.015000e+03,...,2.000000,0.010000,17.000000,139.000000,3.000000,3.000000,4.136000e+01,-8.307000e+01,41.390000,-82.627650
max,2.022080e+05,3.100000e+01,2.359000e+03,2.022080e+05,3.100000e+01,2.359000e+03,9.900000e+08,9.900000e+08,9.900000e+01,2.022000e+03,...,5.000000,2315.000000,4576.000000,820.000000,3749.000000,925.000000,9.710000e+01,1.713661e+02,97.100000,171.366100


### Create tornado specific dataframe

In [9]:
# Create filter to grab only the Tornado related rows
tornado_filter = dfWeather.TOR_F_SCALE.notnull()

In [10]:
# Create a new dataframe of tornado events
dfTornados = dfWeather[tornado_filter]
# Check the first few rows
dfTornados.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,40.0,...,0.0,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,48.0,...,0.0,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,NaN,PUB
2,195007,5,1800,195007,5,1800,NaN,10104927,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,NaN,PUB
3,195007,5,1830,195007,5,1830,NaN,10104928,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,NaN,PUB
4,195007,24,1440,195007,24,1440,NaN,10104929,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,NaN,PUB


In [11]:
# Create a list of selected columns to use as a filter
tornadoColumns = [
    'EVENT_TYPE',
    'STATE',
    'INJURIES_DIRECT',
    'INJURIES_INDIRECT',
    'DEATHS_DIRECT',
    'DEATHS_INDIRECT',
    'DAMAGE_PROPERTY',
    'DAMAGE_CROPS',
    'TOR_F_SCALE',
    'TOR_LENGTH',
    'TOR_WIDTH',
    'BEGIN_DATE_TIME',
    'END_DATE_TIME']

In [12]:
# Use column list to filter the dataframe
dfTor = dfTornados.filter(tornadoColumns)

In [13]:
# Determine which columns have null values
print('Null values count\n\n',dfTor.isnull().sum(),'\n')
print('Count rows\n', len(dfTor))

Null values count

 EVENT_TYPE               0
STATE                    0
INJURIES_DIRECT          0
INJURIES_INDIRECT        0
DEATHS_DIRECT            0
DEATHS_INDIRECT          0
DAMAGE_PROPERTY       9668
DAMAGE_CROPS         16102
TOR_F_SCALE              0
TOR_LENGTH              28
TOR_WIDTH               28
BEGIN_DATE_TIME          0
END_DATE_TIME            0
dtype: int64 

Count rows
 72838


In [14]:
# Remove rows when all potential labels referencing weather impact on people are null
# Not relevant to current dataset but could account for future data
PeopleImpact = [
    'INJURIES_DIRECT',
    'INJURIES_INDIRECT',
    'DEATHS_DIRECT',
    'DEATHS_INDIRECT'
]

dfTor = dfTor.dropna(subset=PeopleImpact, thresh=4)

In [15]:
# Determine which columns have null values
print('Null values count\n\n',dfTor.isnull().sum(),'\n')
print('Count rows\n', len(dfTor))

Null values count

 EVENT_TYPE               0
STATE                    0
INJURIES_DIRECT          0
INJURIES_INDIRECT        0
DEATHS_DIRECT            0
DEATHS_INDIRECT          0
DAMAGE_PROPERTY       9668
DAMAGE_CROPS         16102
TOR_F_SCALE              0
TOR_LENGTH              28
TOR_WIDTH               28
BEGIN_DATE_TIME          0
END_DATE_TIME            0
dtype: int64 

Count rows
 72838


In [16]:
# Remove rows when all potential labels referencing weather impact on property are null
PropertyImpact = [
    'DAMAGE_PROPERTY',
    'DAMAGE_CROPS'
]

dfTor = dfTor.dropna(subset=PropertyImpact, thresh=2)

In [17]:
# Determine which columns have null values
print('Null values count\n\n',dfTor.isnull().sum(),'\n')
print('Count rows\n', len(dfTor))

Null values count

 EVENT_TYPE           0
STATE                0
INJURIES_DIRECT      0
INJURIES_INDIRECT    0
DEATHS_DIRECT        0
DEATHS_INDIRECT      0
DAMAGE_PROPERTY      0
DAMAGE_CROPS         0
TOR_F_SCALE          0
TOR_LENGTH           0
TOR_WIDTH            0
BEGIN_DATE_TIME      0
END_DATE_TIME        0
dtype: int64 

Count rows
 55900


In [18]:
# Convert Property and Crop Damage columns to string to support additional processing
dfTor[['DAMAGE_PROPERTY','DAMAGE_CROPS']] = dfTor[['DAMAGE_PROPERTY','DAMAGE_CROPS']].astype(str)

In [19]:
# Determine last character of the Damage_Property column
dfTor.DAMAGE_PROPERTY.str[-1:].unique()

array(['K', 'M', '0', '3', '6', '5', '1', '7', 'B'], dtype=object)

In [20]:
# Determine last character of the Damage_Crops column
dfTor.DAMAGE_CROPS.str[-1:].unique()

array(['0', 'K', 'M'], dtype=object)

In [21]:
# Define monetize method to convert categorized value to a monetory value based on last element in str
def monetize(value):
    if value.isnumeric():
        return float(value)
    if value[-1:] == 'B':
        return float(value[:-1])*pow(10, 9)
    elif value[-1:] == 'M':
        return float(value[:-1])*pow(10, 6)
    elif value[-1:] == 'K':
        return float(value[:-1])*pow(10, 3)
    return -1 # Determine if any value made it past the nested-if

In [22]:
# Convert property and crop damage to valid number using custom method
# Sum both columns together to create new column of Total Property Damage
dfTor['Total Damage Cost'] = dfTor.DAMAGE_CROPS.map(monetize) + dfTor.DAMAGE_PROPERTY.map(monetize)

In [23]:
# Remove original property and crop damage columns
dfTor = dfTor.drop(columns=['DAMAGE_PROPERTY','DAMAGE_CROPS'])

### Convert ALL F-scale to EF-scale based on academic paper

In [24]:
# Determine the frequency of each tornado scale in the dataset
dfTor.TOR_F_SCALE.value_counts()

F1     13565
F0     12749
EF0     9167
F2      7996
EF1     5948
F3      2568
EF2     1668
F4       931
EFU      657
EF3      428
F5       121
EF4       90
EF5       12
Name: TOR_F_SCALE, dtype: int64

In [25]:
# Drop unknown tornado ratings (EFU)
indexEFU = dfTor[ (dfTor['TOR_F_SCALE'] == 'EFU') ].index
dfTor.drop(indexEFU, inplace=True)

In [26]:
# Define rescale method to convert F-scale to EF-scale throughout the dataset
def rescale(value):
    if value[-1:] == '0':
        return 0
    elif value[-1:] == '1':
        return 1
    elif value[-1:] == '2':
        return 2
    elif value[-1:] == '3':
        return 3
    elif value[-1:] == '4':
        return 4
    elif value[-1:] == '5':
        return 5
    return -1 # Determine if any value made it past the nested-if

In [27]:
# Use rescale method to convert F-scale to EF-scale
dfTor['EF-Scale'] = dfTor.TOR_F_SCALE.map(rescale)

In [28]:
# Remove original F-scale column
dfTor = dfTor.drop(columns=['TOR_F_SCALE'])

In [29]:
# Convert datetime columns from object datatype to a proper datetime
#dfTor.BEGIN_DATE_TIME = pd.to_datetime(dfTor.BEGIN_DATE_TIME, errors='coerce')
#dfTor.END_DATE_TIME = pd.to_datetime(dfTor.END_DATE_TIME, errors='coerce')

In [30]:
# Calculate the reported duration of each tornado in minutes based on start and end time reporting
#dfTor['Duration Minutes'] = (dfTor.END_DATE_TIME - dfTor.BEGIN_DATE_TIME).dt.total_seconds() / 60

In [31]:
# Split dataset into a training set and a test set
TornadoDamage_Train, TornadoDamage_Test = train_test_split(dfTor, test_size=0.3)

In [32]:
# Count training rows
TornadoDamage_Train.shape

(38468, 12)

In [33]:
# Count test rows
TornadoDamage_Test.shape

(16487, 12)

In [34]:
# Save a local copy
#dfTor.to_csv('TornadoDataset.csv',index=False)
#TornadoDamage_Train.to_csv('TornadoDamage_Train.csv',index=False)
#TornadoDamage_Test.to_csv('TornadoDamage_Test.csv',index=False)

In [35]:
dfTor

,EVENT_TYPE,STATE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,TOR_LENGTH,TOR_WIDTH,BEGIN_DATE_TIME,END_DATE_TIME,Total Damage Cost,EF-Scale
0,Tornado,OKLAHOMA,0,0,0,0,3.40,400.0,28-APR-50 14:45:00,28-APR-50 14:45:00,250000.0,3
1,Tornado,TEXAS,0,0,0,0,11.50,200.0,29-APR-50 15:30:00,29-APR-50 15:30:00,25000.0,1
2,Tornado,PENNSYLVANIA,2,0,0,0,12.90,33.0,05-JUL-50 18:00:00,05-JUL-50 18:00:00,25000.0,2
3,Tornado,PENNSYLVANIA,0,0,0,0,0.00,13.0,05-JUL-50 18:30:00,05-JUL-50 18:30:00,2500.0,2
4,Tornado,PENNSYLVANIA,0,0,0,0,0.00,33.0,24-JUL-50 14:40:00,24-JUL-50 14:40:00,2500.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
53542,Tornado,OHIO,0,0,0,0,13.75,150.0,08-JUN-22 16:53:00,08-JUN-22 17:16:00,1000000.0,2
53630,Tornado,WISCONSIN,0,0,0,0,7.51,250.0,15-JUN-22 15:22:00,15-JUN-22 15:34:00,100000.0,2
53641,Tornado,COLORADO,0,0,0,0,4.18,5.0,03-JUN-22 16:23:00,03-JUN-22 16:28:00,5000.0,0
53725,Tornado,NEW MEXICO,0,0,0,0,0.19,10.0,07-JUN-22 16:30:00,07-JUN-22 16:35:00,0.0,0


In [36]:
dfTor.columns

Index(['EVENT_TYPE', 'STATE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'TOR_LENGTH', 'TOR_WIDTH',
       'BEGIN_DATE_TIME', 'END_DATE_TIME', 'Total Damage Cost', 'EF-Scale'],
      dtype='object')

In [37]:
dfWeather.YEAR.unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype=int64)

In [38]:
dfWeather.BEGIN_DATE_TIME.head()

0    28-APR-50 14:45:00
1    29-APR-50 15:30:00
2    05-JUL-50 18:00:00
3    05-JUL-50 18:30:00
4    24-JUL-50 14:40:00
Name: BEGIN_DATE_TIME, dtype: object

In [39]:
dfTor.BEGIN_DATE_TIME.head()

0    28-APR-50 14:45:00
1    29-APR-50 15:30:00
2    05-JUL-50 18:00:00
3    05-JUL-50 18:30:00
4    24-JUL-50 14:40:00
Name: BEGIN_DATE_TIME, dtype: object

In [40]:
# pd.to_datetime(dfWeather['BEGIN_DATE_TIME'].head(), '%D-%m-%y %H:%M')

In [41]:
dfTor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54955 entries, 0 to 53757
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EVENT_TYPE         54955 non-null  object 
 1   STATE              54955 non-null  object 
 2   INJURIES_DIRECT    54955 non-null  int64  
 3   INJURIES_INDIRECT  54955 non-null  int64  
 4   DEATHS_DIRECT      54955 non-null  int64  
 5   DEATHS_INDIRECT    54955 non-null  int64  
 6   TOR_LENGTH         54955 non-null  float64
 7   TOR_WIDTH          54955 non-null  float64
 8   BEGIN_DATE_TIME    54955 non-null  object 
 9   END_DATE_TIME      54955 non-null  object 
 10  Total Damage Cost  54955 non-null  float64
 11  EF-Scale           54955 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 5.5+ MB
